In [3]:
import numpy as np
import cv2

In [1]:
def resnet152_v2_summary(net, layer_type=False, layer_shape=False, 
                         get_flops=False, get_memory=False,
                         get_total_time=False, get_layer_time=False):
    # check if the model is empty
    if net.empty():
        print("Net is empty")
        return
        
    # get the types of layers used in the model
    if layer_type:
        print("\nNet contains:")
        for t in net.getLayerTypes():
            print("\t{} layers of type {}".format(net.getLayersCount(t), t))
        
    
    
    # get the tensor shapes for the loaded model and specified input shape
    if layer_shape:
        layers_ids, in_shapes, out_shapes = net.getLayersShapes([1, 3, 224, 224])
        layers_names = net.getLayerNames()
        print("\nNet layers shapes: ")
        for l in range(len(layers_names)):
            in_num, out_num = len(in_shapes[l]), len(out_shapes[l])
            print("  Layer {} has {} inputs and {} outputs".format(layers_names[l],
                                                                 in_num, out_num))
            for i in range(in_num):
                print("\tinput #{} has shape {}".format(i, in_shapes[l][i].flatten()))
            for i in range(out_num):
                print("\toutput #{} has shape {}".format(i, out_shapes[l][i].flatten()))
            
    # compute the number of FLOPs
    if get_flops:
        print("\ngflops: ", net.getFLOPS((1, 3, 224, 224)) * 1e-9)
    
    # report the amount of memory consumed for storing weights and intermediate tensors
    if get_memory:
        w, b = net.getMemoryConsumption((1, 3, 224, 224))
        print("\nweights (mb):", w * 1e-6, ", blobs (mb):", b * 1e-6)
    
    # perform a forward pass for a mock input
    blob = cv2.dnn.blobFromImage(np.zeros((224, 224, 3), np.uint8), 1, (224, 224))
    net.setInput(blob)
    net.forward()
    
    # report the total time
    if get_total_time:
        total, timings = net.getPerfProfile()
        tick2ms = 1000 / cv2.getTickFrequency()
        print("\ninference (ms): {:2f}".format(total * tick2ms))
    
    # report the per layer inference time
    if get_layer_time:
        layer_names = net.getLayerNames()
        print("\n{: <30} {}".format("LAYER", "TIME (ms)"))
        for (i,t) in enumerate(timings):
            print("{: <30} {:.2f}".format(layer_names[i], t[0] * tick2ms))

In [4]:
model_path = "./model/resnet152v2.onnx"
resnet = cv2.dnn.readNetFromONNX(model_path)

In [5]:
resnet152_v2_summary(resnet, layer_type=True)


Net contains:
	153 layers of type BatchNorm
	155 layers of type Convolution
	50 layers of type Eltwise
	1 layers of type InnerProduct
	2 layers of type Pooling
	152 layers of type Relu
	1 layers of type Reshape
	1 layers of type __NetInputLayer__


In [6]:
resnet152_v2_summary(resnet, layer_shape=True)


Net layers shapes: 
  Layer resnetv27_batchnorm0_fwd has 1 inputs and 1 outputs
	input #0 has shape [  1   3 224 224]
	output #0 has shape [  1   3 224 224]
  Layer resnetv27_conv0_fwd has 1 inputs and 1 outputs
	input #0 has shape [  1   3 224 224]
	output #0 has shape [  1   3 224 224]
  Layer resnetv27_batchnorm1_fwd has 1 inputs and 1 outputs
	input #0 has shape [  1   3 224 224]
	output #0 has shape [  1  64 112 112]
  Layer resnetv27_relu0_fwd has 1 inputs and 1 outputs
	input #0 has shape [  1  64 112 112]
	output #0 has shape [  1  64 112 112]
  Layer resnetv27_pool0_fwd has 1 inputs and 1 outputs
	input #0 has shape [  1  64 112 112]
	output #0 has shape [  1  64 112 112]
  Layer resnetv27_stage1_batchnorm0_fwd has 1 inputs and 1 outputs
	input #0 has shape [  1  64 112 112]
	output #0 has shape [ 1 64 56 56]
  Layer resnetv27_stage1_activation0 has 1 inputs and 1 outputs
	input #0 has shape [ 1 64 56 56]
	output #0 has shape [ 1 64 56 56]
  Layer resnetv27_stage1_conv0_fwd h

	output #0 has shape [  1 256  28  28]
  Layer resnetv27_stage3_conv1_fwd has 1 inputs and 1 outputs
	input #0 has shape [  1 256  28  28]
	output #0 has shape [  1 256  28  28]
  Layer resnetv27_stage3_batchnorm2_fwd has 1 inputs and 1 outputs
	input #0 has shape [  1 256  28  28]
	output #0 has shape [  1 256  14  14]
  Layer resnetv27_stage3_activation2 has 1 inputs and 1 outputs
	input #0 has shape [  1 256  14  14]
	output #0 has shape [  1 256  14  14]
  Layer resnetv27_stage3_conv2_fwd has 1 inputs and 1 outputs
	input #0 has shape [  1 256  14  14]
	output #0 has shape [  1 256  14  14]
  Layer resnetv27_stage3_conv3_fwd has 1 inputs and 1 outputs
	input #0 has shape [  1 256  14  14]
	output #0 has shape [   1 1024   14   14]
  Layer resnetv27_stage3__plus0 has 1 inputs and 1 outputs
	input #0 has shape [  1 512  28  28]
	output #0 has shape [   1 1024   14   14]
  Layer resnetv27_stage3_batchnorm3_fwd has 2 inputs and 1 outputs
	input #0 has shape [   1 1024   14   14]
	input

	input #0 has shape [   1 1024   14   14]
	output #0 has shape [   1 1024   14   14]
  Layer resnetv27_stage3_conv55_fwd has 1 inputs and 1 outputs
	input #0 has shape [   1 1024   14   14]
	output #0 has shape [   1 1024   14   14]
  Layer resnetv27_stage3_batchnorm55_fwd has 1 inputs and 1 outputs
	input #0 has shape [   1 1024   14   14]
	output #0 has shape [  1 256  14  14]
  Layer resnetv27_stage3_activation55 has 1 inputs and 1 outputs
	input #0 has shape [  1 256  14  14]
	output #0 has shape [  1 256  14  14]
  Layer resnetv27_stage3_conv56_fwd has 1 inputs and 1 outputs
	input #0 has shape [  1 256  14  14]
	output #0 has shape [  1 256  14  14]
  Layer resnetv27_stage3_batchnorm56_fwd has 1 inputs and 1 outputs
	input #0 has shape [  1 256  14  14]
	output #0 has shape [  1 256  14  14]
  Layer resnetv27_stage3_activation56 has 1 inputs and 1 outputs
	input #0 has shape [  1 256  14  14]
	output #0 has shape [  1 256  14  14]
  Layer resnetv27_stage3_conv57_fwd has 1 inputs 

	input #0 has shape [  1 256  14  14]
	output #0 has shape [  1 256  14  14]
  Layer resnetv27_stage3_batchnorm104_fwd has 1 inputs and 1 outputs
	input #0 has shape [  1 256  14  14]
	output #0 has shape [  1 256  14  14]
  Layer resnetv27_stage3_activation104 has 1 inputs and 1 outputs
	input #0 has shape [  1 256  14  14]
	output #0 has shape [  1 256  14  14]
  Layer resnetv27_stage3_conv105_fwd has 1 inputs and 1 outputs
	input #0 has shape [  1 256  14  14]
	output #0 has shape [  1 256  14  14]
  Layer resnetv27_stage3__plus34 has 1 inputs and 1 outputs
	input #0 has shape [  1 256  14  14]
	output #0 has shape [   1 1024   14   14]
  Layer resnetv27_stage3_batchnorm105_fwd has 2 inputs and 1 outputs
	input #0 has shape [   1 1024   14   14]
	input #1 has shape [   1 1024   14   14]
	output #0 has shape [   1 1024   14   14]
  Layer resnetv27_stage3_activation105 has 1 inputs and 1 outputs
	input #0 has shape [   1 1024   14   14]
	output #0 has shape [   1 1024   14   14]
  Lay

In [8]:
resnet152_v2_summary(resnet, get_flops=True, get_memory=True)


gflops:  23.165506048

weights (mb): 241.31655999999998 , blobs (mb): 314.824608


In [9]:
resnet152_v2_summary(resnet, get_total_time=True, get_layer_time=True)


inference (ms): 2863.795958

LAYER                          TIME (ms)
resnetv27_batchnorm0_fwd       0.22
resnetv27_conv0_fwd            60.13
resnetv27_batchnorm1_fwd       0.00
resnetv27_relu0_fwd            0.00
resnetv27_pool0_fwd            2.66
resnetv27_stage1_batchnorm0_fwd 0.17
resnetv27_stage1_activation0   0.11
resnetv27_stage1_conv0_fwd     6.50
resnetv27_stage1_batchnorm1_fwd 0.00
resnetv27_stage1_activation1   0.00
resnetv27_stage1_conv1_fwd     17.19
resnetv27_stage1_batchnorm2_fwd 0.00
resnetv27_stage1_activation2   0.00
resnetv27_stage1_conv2_fwd     5.26
resnetv27_stage1_conv3_fwd     13.36
resnetv27_stage1__plus0        1.71
resnetv27_stage1_batchnorm3_fwd 1.31
resnetv27_stage1_activation3   0.89
resnetv27_stage1_conv4_fwd     9.84
resnetv27_stage1_batchnorm4_fwd 0.00
resnetv27_stage1_activation4   0.00
resnetv27_stage1_conv5_fwd     47.45
resnetv27_stage1_batchnorm5_fwd 0.00
resnetv27_stage1_activation5   0.00
resnetv27_stage1_conv6_fwd     6.99
resnetv27_stage1__p